# **Pseudopotential**

### [Go back to index](./index.ipynb)

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

<p style="text-align: justify;font-size:15px">
    This notebook presents how pseudopotentials are constructed. The pseudopotential 
    is an effective potential constructed to replace the complicated effects of the 
    core electrons.
</p>

In [70]:
from sympy.physics.hydrogen import R_nl, E_nl
from sympy.abc import r
from sympy.functions import exp
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
from sympy.solvers.solveset import linsolve
from ipywidgets import FloatSlider, Button
from scipy.optimize import newton
import matplotlib.gridspec as gridspec


%matplotlib widget

In [71]:
n = 4
l = 1

ho = R_nl(4, l, r, 1)
Ea = E_nl(n, l)
rf = lambdify(r, ho, "numpy")

In [72]:
s_rc = FloatSlider(value = 30.0, min = 0.0, max = 70, description = "Cutoff $R_c$ : ", layout={'width':'600px'});
compute = Button(description="Compute pseudopotential");


img = plt.figure(tight_layout=True, figsize=(8,7))
img.canvas.header_visible = False

gs = gridspec.GridSpec(2, 1)

ax1 = img.add_subplot(gs[0, 0])
ax2 = img.add_subplot(gs[1, 0])


x1 = np.arange(0, 70.0, 0.01)
y1 = rf(x1)*x1

ax1.plot(x1, y1, 'r-', label="$\Psi(r)r$")
ax1.fill_between(x1, y1, 0, where=x1<s_rc.value, facecolor='yellow', alpha=0.5)
ax1.set_xlim([0, 70.0])

ax1.hlines(0, 0, 70, 'k','--')

line_rc1 = ax1.axvline(s_rc.value)
line_pswf, = ax1.plot([],[],'b-', linewidth=1.5, label="$\Psi^{PS}(r)r$")
ann_rc = ax1.annotate("$R_c$", xy=(s_rc.value + 1.0, -0.1), fontsize=20)

ax1.set_xlabel("r", fontsize = 15)
ax1.set_ylabel("$\Psi(r)r$", fontsize = 15)

ax1.legend(fontsize=15)


x2 = np.linspace(5.0, 70, 600);
y2 = -1.0/x2;

ax2.plot(x2, y2, 'r-', label="$-Z/r$")

ax2.set_xlim([0, 70.0])
ax2.set_ylim([-0.10, 0.10])
line_rc2 = ax2.axvline(s_rc.value)
line_psv, = ax2.plot([],[], 'b-', linewidth=1.5, label="$V^{SP}(r)$")
ax2.hlines(0, 0, 70, 'k','--')

ax2.set_xlabel("r", fontsize = 15)
ax2.set_ylabel("$V$", fontsize = 15)

ax2.legend(fontsize=15)

plt.show()

def on_rc_change(b):
    line_rc1.set_data(s_rc.value, [-1, 1])
    line_rc2.set_data(s_rc.value, [-1, 1])
    ann_rc.set_position((s_rc.value + 1.0, -0.1))
    ax1.collections.clear()
    ax1.fill_between(x1, y1, 0, where=x1<s_rc.value, facecolor='yellow', alpha=0.5)
    ax1.hlines(0, 0, 70, 'k','--')
    
    
s_rc.observe(on_rc_change, names='value')

display(s_rc)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FloatSlider(value=30.0, description='Cutoff $R_c$ : ', layout=Layout(width='600px'), max=70.0)

In [73]:
def compute_right_derivative(rc):
    k0 = rf(rc)
    k1 = diff(ho, r).subs(r, rc).evalf()
    k2 = diff(ho, r, 2).subs(r, rc).evalf()
    return np.array([float(k0), float(k1), float(k2)])

def compute_left_derivative(ps, rc):
    psf = lambdify(r, ps, "numpy")
    k0 = psf(rc)
    k1 = diff(ps, r).subs(r, rc).evalf()
    k2 = diff(ps, r, 2).subs(r, rc).evalf()
    return np.array([float(k0), float(k1), float(k2)])
    
def solver_kernel(devs, rc, l, b):
    A = np.zeros([3,4])
    A[0, :] = np.array([1, rc**2, rc**3, rc**4]);
    A[1, :] = np.array([0, 2*rc, 3*rc**2, 4*rc**3]);
    A[2, :] = np.array([0, 2, 6*rc, 12*rc**2])
    
    B = np.zeros(3)
    B[0] = log(devs[0]/rc**l);
    B[1] = devs[1]/devs[0] - l/rc;
    B[2] = devs[2]/devs[0] - devs[1]**2/devs[0]**2 + l/rc**2;
    
    B-=b*A[:, 1];
    A = np.delete(A, (1), axis=1);
    
    coff = np.linalg.solve(A, B);
    coff = np.insert(coff, 1, b)
    
    return coff

def diff_norms(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    #norm1 = integrate(ho*ho*r*r, (r, 0, rc)).evalf()
    
    ps  = r**l*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = lambdify(r, ps, "numpy")
    psr = lambdify(r, ps*ps*r*r, "numpy")
    hor = lambdify(r, ho*ho*r*r, "numpy")
        
    x1 = np.linspace(0, rc, 500);
        
    norm1 = np.sum(x1*hor(x1))*(x1[1]-x1[0])
    norm2 = np.sum(x1*psr(x1))*(x1[1]-x1[0])
    
    return float(norm1 - norm2)

def plot_ps_wavefunction(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    ps  = r**l*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = lambdify(r, ps*r, "numpy")
    
    devl = compute_left_derivative(ps, rc)
    
    x1 = np.linspace(0, rc, 800);
    line_pswf.set_data(x1, psf(x1));
    ax1.fill_between(x1, psf(x1), 0, where=x1<s_rc.value, facecolor='green', alpha=0.5)

def plot_ps_potential(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    pf  = r**l*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = Ea - l*(l+1)/(2*r*r) + 1/(2*pf)*diff(pf, r, r)
    psfn = lambdify(r, psf, "numpy")
        
    x1 = np.linspace(0.1, rc, 800);
    line_psv.set_data(x1, psfn(x1));
    

def compute_pseudopotential(c):
    b = newton(lambda x: diff_norms(x, s_rc.value, l), x0 = 0.0)
    plot_ps_wavefunction(b, s_rc.value, l)
    plot_ps_potential(b, s_rc.value, l)
    
compute.on_click(compute_pseudopotential)

display(compute)

Button(description='Compute pseudopotential', style=ButtonStyle())